In [1]:
import pandas as pd
import pyproj
import numpy as np
import datetime

In [2]:
complaints = pd.read_csv('NYPD_Complaint_Map_Historic_.csv', usecols=(0, 1, 2, 7, 10, 13, 19, 20, 21, 22), thousands=',' )
subways = pd.read_csv('NYC_Subway_Entrances.csv')

In [3]:
print(complaints.info())
print('-----------------------------')
print(complaints.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5580035 entries, 0 to 5580034
Data columns (total 10 columns):
CMPLNT_NUM          int64
CMPLNT_FR_DT        object
CMPLNT_FR_TM        object
OFNS_DESC           object
CRM_ATPT_CPTD_CD    object
BORO_NM             object
X_COORD_CD          float64
Y_COORD_CD          float64
Latitude            float64
Longitude           float64
dtypes: float64(4), int64(1), object(5)
memory usage: 425.7+ MB
None
-----------------------------
   CMPLNT_NUM CMPLNT_FR_DT CMPLNT_FR_TM OFNS_DESC CRM_ATPT_CPTD_CD BORO_NM  \
0   101109527   12/31/2015     23:45:00   FORGERY        COMPLETED   BRONX   

   X_COORD_CD  Y_COORD_CD   Latitude  Longitude  
0   1007314.0    241257.0  40.828848 -73.916661  


In [4]:
complaints['CMPLNT_FR_DT'] = pd.to_datetime(complaints['CMPLNT_FR_DT'], format = '%m/%d/%Y', errors='coerce' )
complaints['CMPLNT_FR_TM'] = pd.to_timedelta(complaints['CMPLNT_FR_TM'], errors='coerce' )
print(complaints.info())
print(complaints.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5580035 entries, 0 to 5580034
Data columns (total 10 columns):
CMPLNT_NUM          int64
CMPLNT_FR_DT        datetime64[ns]
CMPLNT_FR_TM        timedelta64[ns]
OFNS_DESC           object
CRM_ATPT_CPTD_CD    object
BORO_NM             object
X_COORD_CD          float64
Y_COORD_CD          float64
Latitude            float64
Longitude           float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(3), timedelta64[ns](1)
memory usage: 425.7+ MB
None
   CMPLNT_NUM CMPLNT_FR_DT CMPLNT_FR_TM OFNS_DESC CRM_ATPT_CPTD_CD BORO_NM  \
0   101109527   2015-12-31     23:45:00   FORGERY        COMPLETED   BRONX   

   X_COORD_CD  Y_COORD_CD   Latitude  Longitude  
0   1007314.0    241257.0  40.828848 -73.916661  


In [5]:
for col in complaints:
    print(complaints[col].isnull().values.sum())

0
684
48
18892
7
463
195868
195868
195868
195868


In [6]:
complaints = complaints.dropna(subset=[['X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude']])
for col in complaints:
    print(complaints[col].isnull().values.sum())
print('--------------------')
print(complaints.info())
print(complaints[['X_COORD_CD', 'Y_COORD_CD']].head(5))

0
661
46
18152
6
75
0
0
0
0
--------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5384167 entries, 0 to 5580034
Data columns (total 10 columns):
CMPLNT_NUM          int64
CMPLNT_FR_DT        datetime64[ns]
CMPLNT_FR_TM        timedelta64[ns]
OFNS_DESC           object
CRM_ATPT_CPTD_CD    object
BORO_NM             object
X_COORD_CD          float64
Y_COORD_CD          float64
Latitude            float64
Longitude           float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(3), timedelta64[ns](1)
memory usage: 451.9+ MB
None
   X_COORD_CD  Y_COORD_CD
0   1007314.0    241257.0
1   1043991.0    193406.0
2    999463.0    231690.0
3   1060183.0    177862.0
4    987606.0    208148.0


In [7]:
#Now to reduce entries even further so there are not roughly 5.4 million. Let's focus on the top 10 
#Crimes only from 2016
complaints_2016 = complaints[(complaints['CMPLNT_FR_DT'] > datetime.date(2016,1,1)) & (complaints['CMPLNT_FR_DT'] 
                             < datetime.date(2016,12,31))]
#Crimes that occured in Manhattan. First we need to find out how the boro names are stored 
print(complaints_2016['BORO_NM'].unique())

['BRONX' 'QUEENS' 'MANHATTAN' 'BROOKLYN' 'STATEN ISLAND']


In [8]:
#Run the command to only use records that occur in Manhattan
complaints_2016_man = complaints_2016[complaints_2016.BORO_NM == 'MANHATTAN']

#Check new information
print(complaints_2016_man.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110387 entries, 5101986 to 5578367
Data columns (total 10 columns):
CMPLNT_NUM          110387 non-null int64
CMPLNT_FR_DT        110387 non-null datetime64[ns]
CMPLNT_FR_TM        110387 non-null timedelta64[ns]
OFNS_DESC           110382 non-null object
CRM_ATPT_CPTD_CD    110387 non-null object
BORO_NM             110387 non-null object
X_COORD_CD          110387 non-null float64
Y_COORD_CD          110387 non-null float64
Latitude            110387 non-null float64
Longitude           110387 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(3), timedelta64[ns](1)
memory usage: 9.3+ MB
None


In [9]:
#Use only top occuring crimes
print(complaints_2016_man['OFNS_DESC'].value_counts())

PETIT LARCENY                           24182
GRAND LARCENY                           15697
HARRASSMENT 2                           12952
ASSAULT 3 & RELATED OFFENSES            10255
CRIMINAL MISCHIEF & RELATED OF           9696
DANGEROUS DRUGS                          6572
OFF. AGNST PUB ORD SENSBLTY &            4527
FELONY ASSAULT                           3714
ROBBERY                                  2888
OFFENSES AGAINST PUBLIC ADMINI           2391
BURGLARY                                 2382
DANGEROUS WEAPONS                        2288
MISCELLANEOUS PENAL LAW                  1967
FORGERY                                  1516
CRIMINAL TRESPASS                        1352
THEFT-FRAUD                              1266
VEHICLE AND TRAFFIC LAWS                 1072
INTOXICATED & IMPAIRED DRIVING            878
FRAUDS                                    815
GRAND LARCENY OF MOTOR VEHICLE            720
OFFENSES INVOLVING FRAUD                  550
POSSESSION OF STOLEN PROPERTY     

In [10]:
#Let's only use the top 10 offenses.
list_10 = ['PETIT LARCENY', 'GRAND LARCENY','HARRASSMENT 2', 'ASSAULT 3 & RELATED OFFENSES', 'CRIMINAL MISCHIEF & RELATED OF',
          'DANGEROUS DRUGS', 'OFF. AGNST PUB ORD SENSBLTY &', 'FELONY ASSAULT', 'ROBBERY']

complaints_10 = complaints_2016_man[complaints_2016_man.OFNS_DESC.isin(list_10)]

print(complaints_10.info())

#Let's reset the index of the crimes so it starts at 0
complaints_10 = complaints_10.reset_index()

print(complaints_10.head(2))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90483 entries, 5101986 to 5578367
Data columns (total 10 columns):
CMPLNT_NUM          90483 non-null int64
CMPLNT_FR_DT        90483 non-null datetime64[ns]
CMPLNT_FR_TM        90483 non-null timedelta64[ns]
OFNS_DESC           90483 non-null object
CRM_ATPT_CPTD_CD    90483 non-null object
BORO_NM             90483 non-null object
X_COORD_CD          90483 non-null float64
Y_COORD_CD          90483 non-null float64
Latitude            90483 non-null float64
Longitude           90483 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(3), timedelta64[ns](1)
memory usage: 7.6+ MB
None
     index  CMPLNT_NUM CMPLNT_FR_DT CMPLNT_FR_TM      OFNS_DESC  \
0  5101986   190600818   2016-12-30     23:50:00        ROBBERY   
1  5101997   475992375   2016-12-30     23:30:00  GRAND LARCENY   

  CRM_ATPT_CPTD_CD    BORO_NM  X_COORD_CD  Y_COORD_CD   Latitude  Longitude  
0        COMPLETED  MANHATTAN    988225.0    198162.0  40

In [11]:
print(subways.info())
print('-----------------------------')
print(subways.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 5 columns):
OBJECTID    1928 non-null int64
URL         1928 non-null object
NAME        1899 non-null object
the_geom    1928 non-null object
LINE        1928 non-null object
dtypes: int64(1), object(4)
memory usage: 75.4+ KB
None
-----------------------------
   OBJECTID                                URL  \
0      1734  http://web.mta.info/nyct/service/   

                                      NAME  \
0  Birchall Ave & Sagamore St at NW corner   

                                       the_geom LINE  
0  POINT (-73.86835600032798 40.84916900104506)  2-5  


In [12]:
subways_loc = pd.DataFrame(columns=['ID', 'geom'])
subways_loc[['ID', 'geom']] = subways[['OBJECTID', 'the_geom']]
print (subways_loc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 2 columns):
ID      1928 non-null int64
geom    1928 non-null object
dtypes: int64(1), object(1)
memory usage: 30.2+ KB
None


In [13]:
subways_loc = pd.DataFrame(subways_loc.geom.str.split(' ').tolist(),
                                   columns = ['point','Longitude', 'Latitude'])
print (subways_loc.head(1))

   point            Longitude            Latitude
0  POINT  (-73.86835600032798  40.84916900104506)


In [14]:
subways_loc['Longitude'] = subways_loc['Longitude'].map(lambda x: str(x)[1:])
subways_loc['Latitude'] = subways_loc['Latitude'].map(lambda x: str(x)[:-1])
print (subways_loc.head(1))

   point           Longitude           Latitude
0  POINT  -73.86835600032798  40.84916900104506


In [15]:
subways_loc = subways_loc[['Latitude', 'Longitude']]
subways_loc[['Latitude', 'Longitude']] = subways_loc[['Latitude', 'Longitude']].astype(float)
print (subways_loc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 2 columns):
Latitude     1928 non-null float64
Longitude    1928 non-null float64
dtypes: float64(2)
memory usage: 30.2 KB
None


In [16]:
WGS84_Z18 = pyproj.Proj(init = "EPSG:4326", preserve_units=True)
NYS_LIZ = pyproj.Proj(init = "ESRI:102718", preserve_units=True)

In [17]:
def coord_transform(df):
    x = pd.Series()
    y = pd.Series()
    for idx, val in enumerate(df['Latitude']):
        x1, y1 = pyproj.transform(WGS84_Z18, NYS_LIZ, df['Longitude'][idx], df['Latitude'][idx])
        x.set_value(idx, x1)
        y.set_value(idx, y1)
    df['x'] = x
    df['y'] = y
    return df
    
subways_loc_trans = coord_transform(subways_loc)

#Check to make sure all entries look correct
print(subways_loc_trans.head(5))
print(subways_loc_trans.info())

    Latitude  Longitude             x              y
0  40.849169 -73.868356  1.020671e+06  248679.990550
1  40.849128 -73.868213  1.020710e+06  248665.112299
2  40.841223 -73.873499  1.019252e+06  245782.869704
3  40.841453 -73.872892  1.019420e+06  245866.910219
4  40.840815 -73.879623  1.017558e+06  245631.832277
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 4 columns):
Latitude     1928 non-null float64
Longitude    1928 non-null float64
x            1928 non-null float64
y            1928 non-null float64
dtypes: float64(4)
memory usage: 60.3 KB
None


In [18]:
#Need to split up the crimes dataframe so the distance analysis can run without crashing my computer
crime1, crime2, crime3, crime4, crime5 = np.array_split(complaints_10, 5)

#Check one of the new dfs to make sure it keeps the same index as the original. This is important when merging the distances
#later
print(crime3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18097 entries, 36194 to 54290
Data columns (total 11 columns):
index               18097 non-null int64
CMPLNT_NUM          18097 non-null int64
CMPLNT_FR_DT        18097 non-null datetime64[ns]
CMPLNT_FR_TM        18097 non-null timedelta64[ns]
OFNS_DESC           18097 non-null object
CRM_ATPT_CPTD_CD    18097 non-null object
BORO_NM             18097 non-null object
X_COORD_CD          18097 non-null float64
Y_COORD_CD          18097 non-null float64
Latitude            18097 non-null float64
Longitude           18097 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(2), object(3), timedelta64[ns](1)
memory usage: 1.5+ MB
None


In [19]:
#set up column for the minimum distance between a crime and the closest subway station
complaints_10['min_dist'] = np.nan

In [20]:
def min_dist_func(crime_df):
    df_new = pd.DataFrame(index = crime_df.index.copy(), columns = subways_loc_trans.index.copy())
    df_new['min_dist'] = np.nan
    
    for idx_crime, x_crime in enumerate(crime_df['X_COORD_CD']):
        y_crime = crime_df['Y_COORD_CD'].iloc[idx_crime]
        for idx_subway, x_subway in enumerate(subways_loc_trans['x']):
            y_subway = subways_loc_trans['y'].iloc[idx_subway]
            
            dist = np.sqrt((x_crime - x_subway)**2 + (y_crime - y_subway)**2)
            df_new.iloc[idx_crime, idx_subway] = dist
            
    return df_new

In [21]:
#run min_dist_list with each smaller crime df to obtain lists.
min_dist1 = pd.read_csv('min_dist1.csv')
min_dist2 = pd.read_csv('min_dist2.csv')
min_dist3 = pd.read_csv('min_dist3.csv')
min_dist4 = pd.read_csv('mindist4.csv')
min_dist5 = pd.read_csv('mindist5.csv')

#run min_dist_list with each smaller crime df to obtain lists.
min_dist1 = min_dist_func(crime1)
min_dist2 = min_dist_func(crime2)
min_dist3 = min_dist_func(crime3)
min_dist4 = min_dist_func(crime4)
min_dist5 = min_dist_func(crime5)
min_dist6 = min_dist_func(crime6) 

In [22]:
#Extra column in front that has index numbers that should be removed for the min function to work properly
min_dist1 = min_dist1.drop(min_dist1.columns[0], axis=1)
min_dist2 = min_dist2.drop(min_dist2.columns[0], axis=1)
min_dist3 = min_dist3.drop(min_dist3.columns[0], axis=1)
min_dist4 = min_dist4.drop(min_dist4.columns[0], axis=1)
min_dist5 = min_dist5.drop(min_dist5.columns[0], axis=1)

In [23]:
print(min_dist1.head(2))
print('----------------')
print(min_dist4.head(2))

              0             1             2             3             4  \
0  60039.893248  60048.780567  56836.793693  56998.904505  55801.375809   
1  56613.185604  56628.670352  53519.186668  53695.160560  52265.410354   

              5             6             7             8            9  \
0  55627.706942  55628.016908  55756.807774  55915.764925  53508.59137   
1  52091.489848  52101.566815  52230.221773  52378.512773  49770.99929   

     ...             1919          1920          1921          1922  \
0    ...     21746.666295  26277.216938  26280.271461  26221.933542   
1    ...     17490.293108  21929.320298  21918.007505  21853.966317   

           1923          1924          1925          1926          1927  \
0  25575.753139  28798.075890  28721.289486  28374.177927  28336.698503   
1  21216.141172  24343.360018  24266.659246  23976.051940  23893.948188   

   min_dist  
0       NaN  
1       NaN  

[2 rows x 1929 columns]
----------------
              0            

In [24]:
#Find the minimum distance between each crime and the subway stations.
min_dist1['min_dist'] = min_dist1.min(axis=1)
min_dist2['min_dist'] = min_dist2.min(axis=1)
min_dist3['min_dist'] = min_dist3.min(axis=1)
min_dist4['min_dist'] = min_dist4.min(axis=1)
min_dist5['min_dist'] = min_dist5.min(axis=1)

In [25]:
print(min_dist1['min_dist'].head())
print('----------------')
print(min_dist4['min_dist'].head())

0    1463.107789
1     644.490852
2     401.105386
3     870.446335
4     169.878256
Name: min_dist, dtype: float64
----------------
0     657.130081
1    1120.531803
2     338.229271
3     371.416604
4    1048.321774
Name: min_dist, dtype: float64


In [26]:
#Concatenate all the min dist values and put into original dataframe
min_dist =  pd.concat([min_dist1['min_dist'], min_dist2['min_dist'], min_dist3['min_dist'], 
                                        min_dist4['min_dist'], min_dist5['min_dist']])

In [27]:
#In order to not throw an "cannot reindex from a duplicate axis" error, I converted the min_dist to values
complaints_10['min_dist'] = min_dist.values

In [28]:
#Check head, tail, and info on complaints_10 to make sure everything populated.
print(complaints_10.head())
print('----------------')
print(complaints_10.tail())
print('----------------')
print(complaints_10.info())

     index  CMPLNT_NUM CMPLNT_FR_DT CMPLNT_FR_TM  \
0  5101986   190600818   2016-12-30     23:50:00   
1  5101997   475992375   2016-12-30     23:30:00   
2  5102004   952833671   2016-12-30     23:15:00   
3  5102006   212105306   2016-12-30     23:00:00   
4  5102014   344963119   2016-12-30     22:50:00   

                      OFNS_DESC CRM_ATPT_CPTD_CD    BORO_NM  X_COORD_CD  \
0                       ROBBERY        COMPLETED  MANHATTAN    988225.0   
1                 GRAND LARCENY        COMPLETED  MANHATTAN    982746.0   
2                 PETIT LARCENY        COMPLETED  MANHATTAN    987703.0   
3  ASSAULT 3 & RELATED OFFENSES        COMPLETED  MANHATTAN    986067.0   
4                 HARRASSMENT 2        COMPLETED  MANHATTAN    995159.0   

   Y_COORD_CD   Latitude  Longitude     min_dist  
0    198162.0  40.710593 -73.985664  1463.107789  
1    206647.0  40.733883 -74.005428   644.490852  
2    201531.0  40.719840 -73.987545   401.105386  
3    199328.0  40.713794 -73.993

In [29]:
#In order to be able to sort in chronological order, it would be best to combine the date and time before exporting
date_time = pd.to_datetime(complaints_10.CMPLNT_FR_DT) + pd.to_timedelta(complaints_10.CMPLNT_FR_TM)
complaints_10['date_time'] = date_time
complaints_10 = complaints_10.drop(complaints_10.columns[[0,2, 3]], axis=1)

In [30]:
#Reset index to date_time
complaints_10.set_index('date_time', drop=True, append=False, inplace=True, verify_integrity=False)

In [31]:
complaints_10 = complaints_10.sort_index()
print(complaints_10.head())

                     CMPLNT_NUM                       OFNS_DESC  \
date_time                                                         
2016-01-02 00:00:00   408593442                   HARRASSMENT 2   
2016-01-02 00:00:00   775874632                 DANGEROUS DRUGS   
2016-01-02 00:00:00   647885883                   GRAND LARCENY   
2016-01-02 00:01:00   961716811  CRIMINAL MISCHIEF & RELATED OF   
2016-01-02 00:05:00   217870969                   PETIT LARCENY   

                    CRM_ATPT_CPTD_CD    BORO_NM  X_COORD_CD  Y_COORD_CD  \
date_time                                                                 
2016-01-02 00:00:00        COMPLETED  MANHATTAN    992729.0    216866.0   
2016-01-02 00:00:00        COMPLETED  MANHATTAN    987472.0    214939.0   
2016-01-02 00:00:00        COMPLETED  MANHATTAN    991504.0    227286.0   
2016-01-02 00:01:00        ATTEMPTED  MANHATTAN    985320.0    199453.0   
2016-01-02 00:05:00        COMPLETED  MANHATTAN    998559.0    242439.0   

    

In [32]:
#export df to use in future analysis
complaints_10.to_csv('crime_complaints.csv')